In [8]:
import pysftp
import os 
import numpy as np
import pandas as pd
import netCDF4
import numpy as np

from scipy import io
from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from dotenv import load_dotenv   #for python-dotenv method
from datetime import datetime

pd.options.mode.chained_assignment = None  # default='warn'


In [9]:
# Get Temperature Data]
def generate_dataset(path='/Users/joaojesus/Desktop/final_year_proj/PROCESSED_ERA5_ONLINE/COMPRESSED/WIND_EASTWARD/',
                    geo_filtering = {'min_lat':60,'max_lat':70,'min_lon':-180, 'max_lon': 180},
                    year_selection = '2018',
                    data_name = 'wind_eastward'):
    
    grid = map_grid(grid_size=None, source = 'era5')
    wizard = data_wizard()
    res_mean = []
    res_mean_selection = []
    file_names = []
    files = os.listdir(path)
    files.sort()
    for file in files:
        if file[:4] == year_selection:
            print(file)
            temperature = np.load(path + file)['arr_0']
            # temperature = np.loadtxt(path + file, delimiter=",")

            select_area_indexer = grid.select_area_indexer(
                min_lat= geo_filtering['min_lat'],
                max_lat= geo_filtering['max_lat'],
                min_lon= geo_filtering['min_lon'],
                max_lon= geo_filtering['max_lat'])

            data = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = temperature)

            file_names.append(file[:10])
            res_mean.append(np.nanmean(temperature))
            res_mean_selection.append(np.nanmean(data))

    df = pd.DataFrame(data={
        'dates' : file_names,
        f'{data_name}_mean' : res_mean,
        f'{data_name}_mean_selection' : res_mean_selection
    })
    
    df['dates'] = pd.to_datetime(df['dates'])
    df = df.sort_values(by='dates')
    df = df.reset_index()
    df = df.drop(columns=['index'])

    return df

def find_ssw(df):
    values = df['wind_eastward_mean_selection'].to_list()
    SSW_ = [False]

    for i in range(1,len(values)):
        if values[i-1] >= 0 and values[i] < 0:
            SSW_.append(True)
        else:
            SSW_.append(False)

    df['SSW_'] = SSW_
    df['SSW_flag'] = False

    day_threshold = 5
    for index in df[df['SSW_'] == True].index:
        if len(df.iloc[index:index+day_threshold].loc[df['wind_eastward_mean_selection'] < 0]) >= day_threshold :
            df['SSW_flag'].iloc[index] = True
    return df
    

In [15]:

for year in range(2006,2007):
    YEAR = year
    print(YEAR)
    df_wind = generate_dataset(path='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/WIND_EASTWARD/',
                        geo_filtering = {'min_lat':60,'max_lat':70,'min_lon':-180, 'max_lon': 180},
                        year_selection = f'{YEAR}',
                        data_name = 'wind_eastward')

    df_temperature = generate_dataset(path='/Users/joaojesus/Desktop/final_year_proj/pre_processed_era5_online/COMPRESSED/TEMPERATURE/',
                        geo_filtering = {'min_lat':55,'max_lat':65,'min_lon':-180, 'max_lon': 180},
                        year_selection = f'{YEAR}',
                        data_name = 'temperature')

    df = pd.merge(df_wind,df_temperature[['dates','temperature_mean_selection']],how='left', on='dates')

    df_ssw = find_ssw(df)
    # find the index of the row where col2 is equal to 7
    index = df.index[df['SSW_flag'] == True].tolist()

    SAVE_SSW_EVENT = True
    if SAVE_SSW_EVENT:
        if len(index) > 0:
            for index in index:
                day_window = 15
                df_dataset = df.loc[index-day_window:index+day_window-10]
                date_str = df.loc[index]['dates'].strftime("%Y-%m-%d")
                df_dataset.to_csv(f'ssw_events_saved/ssw_t0_{date_str}.csv', sep=',',index=False)


2006
2006-01-01.npz
2006-01-02.npz
2006-01-03.npz
2006-01-04.npz
2006-01-05.npz
2006-01-06.npz
2006-01-07.npz
2006-01-08.npz
2006-01-09.npz
2006-01-10.npz
2006-01-11.npz
2006-01-12.npz
2006-01-13.npz
2006-01-14.npz
2006-01-15.npz
2006-01-16.npz
2006-01-17.npz
2006-01-18.npz
2006-01-19.npz
2006-01-20.npz
2006-01-21.npz
2006-01-22.npz
2006-01-23.npz
2006-01-24.npz
2006-01-25.npz
2006-01-26.npz
2006-01-27.npz
2006-01-28.npz
2006-01-29.npz
2006-01-30.npz
2006-01-31.npz
2006-02-01.npz
2006-02-02.npz
2006-02-03.npz
2006-02-04.npz
2006-02-05.npz
2006-02-06.npz
2006-02-07.npz
2006-02-08.npz
2006-02-09.npz
2006-02-10.npz
2006-02-11.npz
2006-02-12.npz
2006-02-13.npz
2006-02-14.npz
2006-02-15.npz
2006-02-16.npz
2006-02-17.npz
2006-02-18.npz
2006-02-19.npz
2006-02-20.npz
2006-02-21.npz
2006-02-22.npz
2006-02-23.npz
2006-02-24.npz
2006-02-25.npz
2006-02-26.npz
2006-02-27.npz
2006-02-28.npz
2006-03-01.npz
2006-03-02.npz
2006-03-03.npz
2006-03-04.npz
2006-03-05.npz
2006-03-06.npz
2006-03-07.npz
2006-

In [16]:
import plotly.graph_objects as go

# df = df_dataset
# Create figure and plot the DataFrame
fig = go.Figure()

# fig.add_trace(go.Scatter(x=df['dates'], y=df['res_filtered'].rolling(3).mean(), name='Res Filtered', mode='markers', marker=dict(color='blue', size=2)))
fig.add_trace(go.Scatter(x=df['dates'], y=df['wind_eastward_mean_selection'], name='wind_eastward_mean_selection', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=df['dates'], y=df['temperature_mean_selection'], name='Temperature Selection Mean', mode='markers', marker=dict(color='orange', size=3), yaxis='y2'))
fig.add_trace(go.Scatter(x=df['dates'], y=df['temperature_mean_selection'].rolling(1).mean(), name='Temperature Selection Mean', line=dict(color='orange'), yaxis='y2'))

# Add a dashed line at y=0
fig.add_shape(type='line', x0=df['dates'].min(), y0=0, x1=df['dates'].max(), y1=0, line=dict(color='gray', dash='dash'))

# Find the index of the first negative value
first_negative_index = (df['wind_eastward_mean_selection'] < 0).idxmin()

# Add vertical lines for SSW_flag == True
for i, row in df.iterrows():
    if row.SSW_flag == True:
        fig.add_shape(type='line', x0=row.dates, y0=-50, x1=row.dates, y1=50, line=dict(color='red', dash='dash'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-50, 80],
        title='wind_eastward_mean_selection'
    )
    ,yaxis2=dict(
        range=[190, 250],
        title='Temperature Selection Mean',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude in {YEAR}',
    legend=dict(orientation="h", y=1.2, x=0.5)
)

# Show the plot
fig.show()